In [7]:
import requests
import time
import random
import os
import re
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import json

In [ ]:
headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "es-ES,es;q=0.9",
    "Cache-Control": "max-age=0",
    "cookie": 'didomi_token=eyJ1c2VyX2lkIjoiMTkzNzc5YmQtZjc3MC02ZWZjLTgzMWItYzhkOTE2YjM3NDNmIiwiY3JlYXRlZCI6IjIwMjQtMTEtMjlUMTE6MDg6NDQuMjc5WiIsInVwZGF0ZWQiOiIyMDI0LTExLTI5VDExOjA4OjUwLjgyNloiLCJ2ZW5kb3JzIjp7ImVuYWJsZWQiOlsiZ29vZ2xlIiwiYzpnb29nbGVhbmEtNFRYbkppZ1IiLCJjOmFkZXZpbnRhLW1vdG9yLW1vdG9yIiwiYzptZXRhcGxhdGYtTlJlVnBtTFEiLCJjOmNyaXRlby1QNDhlR1QydyIsImM6Z29vZ2xlaXJlLWZmS2FQYVJEIl19LCJwdXJwb3NlcyI6eyJlbmFibGVkIjpbImRldmljZV9jaGFyYWN0ZXJpc3RpY3MiLCJnZW9sb2NhdGlvbl9kYXRhIiwidHJhbnNmZXItdG8tbW90b3IiLCJ1c29kZWxvcy1RcExwTThqVyJdfSwidmVuZG9yc19saSI6eyJlbmFibGVkIjpbImdvb2dsZSJdfSwidmVyc2lvbiI6Mn0=; euconsent-v2=CQI2igAQI2igAAHABBENBRFsAP_gAEPgAAiQKoNX_G__bWlr8X73aftkeY1P9_h77sQxBhfJE-4FzLvW_JwXx2ExNA36tqIKmRIAu3TBIQNlGJDURVCgaogVryDMaEyUoTNKJ6BkiFMRM2dYCFxvm4tj-QCY5vr991dx2B-t7dr83dzyy4xHn3a5_2S0WJCdA5-tDfv9bROb-9IOd_x8v4v4_F_pE2_eT1l_tWvp7D9-cts__XW89_fff_9Pn_-uB_-_3_vf_BU4Akw0KiAMsCQkINAwggQAqCsICKBAEAACQNEBACYMCnYGAC6wkQAgBQADBACAAEGQAIAAAIAEIgAgAKBAABAIFAAGABAMBAAQMAAIALAQCAAEB0DFMCCAQLABIzIiFMCEIBIICWyoQSAIEFcIQizwCIBETBQAAAAAFIAAgLBYHEkgJUJBAFxBtAAAQAIBBAAUIJOTAAEAZstQeDBtGVpgGD5gkQ0wDIAiCMgAAAAA.f_wACHwAAAAA; borosTcf=eyJwb2xpY3lWZXJzaW9uIjoyLCJjbXBWZXJzaW9uIjoxLCJwdXJwb3NlIjp7ImNvbnNlbnRzIjp7IjEiOnRydWUsIjIiOnRydWUsIjMiOnRydWUsIjQiOnRydWUsIjUiOnRydWUsIjYiOnRydWUsIjciOnRydWUsIjgiOnRydWUsIjkiOnRydWUsIjEwIjp0cnVlfX0sInNwZWNpYWxGZWF0dXJlcyI6eyIxIjp0cnVlfX0=; kanirante-adv-aauid=d9e6cd6c-f835-4f01-803d-9911b5673f8d; AMCVS_05FF6243578784B37F000101%40AdobeOrg=1; re_uuid=8353b954-879c-4116-ba45-17c4ba2493c2; reese84=3:s/1054mSZFsnpATqluc0Kg==:oCCPjASTlMPOPWHoPtGA0tE9XHS3Dv6ytWBPVydzQGf008owgC4fS8f7Fhte2UBwTx2QcuWF+9rDZDrNIYo7jeg0dezu488V1n0aVs4AGs0W/aC1bp8pjnIJxypOvaRkbHwQ5XG/cNXib4OOI/KyO8Mb6K0bT6DN96UuIYaynweijH7Lh5+CkSao/NovQcxdJRdUNMPJflMu1+x/rAsIfyRuwH7eDsm0k8FLP5O608tnF3SnP0BYDjDUP2BvTQHkznMuI6OMhTrKMg9RGF6MFrpJ+d+Z/uG12yfI+VVbkuaOUd8G+V6B5WCP1x3nHzrVSJ9xTiDaCRoXsyirb7AoSfHq5zyuiQ/TY7WIsvnWU/lOOdWDqAsyN4eMoNs1f9tBG+2gyoLPG5d0eO4qxzDqzRYKaYZM1sWgsiMOT+unOjlVbVjbqsyNgzEPWwSc/FxbJyesrQ/P7+y24oz23ftMoA==:YroThCN6UdkLBbKek9HHAoKeTKouGwTsX8UFWnsFSSY=; AMCV_05FF6243578784B37F000101%40AdobeOrg=-408604571%7CMCIDTS%7C20057%7CMCMID%7C04817536139526365941429215584631391982%7CMCAID%7CNONE%7CMCOPTOUT-1732891447s%7CNONE%7CvVersion%7C4.6.0',
    "Referer": "https://www.idealista.com/alquiler-viviendas/murcia-murcia/",
    "Sec-Ch-Device-Memory": "4",
    "Sec-Ch-Ua": '"Not)A;Brand";v="99", "Google Chrome";v="127", "Chromium";v="127"',
    "Sec-Ch-Ua-Full-Version-List": '"Not)A;Brand";v="99.0.0.0", "Google Chrome";v="127.0.6533.120", "Chromium";v="127.0.6533.120"',
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-User": "?1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36",
    "X-Height": "9999",  # Establece el valor de height
    "X-Width": "1920",   # Puedes agregar otros parámetros, como width
    "X-Depth": "500",    # Ejemplo de parámetro depth, si es necesario
    "X-Color": "blue",   # Ejemplo de parámetro de color, si lo deseas
    "X-Orientation": "landscape"  # Otro ejemplo, si necesitas un parámetro de orientación
}


In [9]:
# Crear una sesión
session = requests.Session()
session.headers.update(headers)

# URL base de Fotocasa
base_url = "https://www.fotocasa.es/es/alquiler/viviendas/murcia-provincia/todas-las-zonas/"

# Paginación y almacenamiento de resultados
page = 1
all_listings = []

while True:
    try:
        print(f"Procesando página {page}...")
        # Realizar la solicitud a la página actual
        response = session.get(f"{base_url}l/{page}")
        response.raise_for_status()  # Verificar que la solicitud sea exitosa

        # Analizar el contenido HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # Encontrar todas las tarjetas de inmuebles
        class_card = "re-CardPackPremium"
        # Parece que a partir de la página 2 cambia la clase?
        if (page > 1):
            class_card = 're-CardPackMinimal'
        listings = soup.find_all("article", class_=class_card)

        # Si no hay más tarjetas, salir del bucle
        if not listings:
            print("No se encontraron más inmuebles. Fin de la paginación.")
            break

        # Almacenar las tarjetas
        for listing in listings:
            try:
                link_tag = listing.find('a', class_='re-CardPackPremium-carousel')
                href = link_tag.get('href') if link_tag else None
                full_link = f"https://www.fotocasa.es{href}" if href and href.startswith("/") else None

                titulo = listing.find("span", class_="re-I18nPropertyTitle").get_text()
                precio = listing.find('span', class_="re-CardPrice").get_text(strip=True)

                features = listing.find("ul", class_="re-CardFeaturesWithIcons-wrapper")
                habitaciones = features.find("span", class_="re-CardFeaturesWithIcons-feature-icon--rooms").get_text(strip=True) if features else "N/A"
                banos = features.find("span", class_="re-CardFeaturesWithIcons-feature-icon--bathrooms").get_text(strip=True) if features else "N/A"
                superficie = features.find("span", class_="re-CardFeaturesWithIcons-feature-icon--surface").get_text(strip=True) if features else "N/A"
                planta = features.find("span", class_="re-CardFeaturesWithIcons-feature-icon--floor").get_text(strip=True) if features else "N/A"

                # Guardar datos del listado
                all_listings.append({
                    "Título": titulo,
                    "Precio": precio,
                    "Habitaciones": habitaciones,
                    "Baños": banos,
                    "Superficie": superficie,
                    "Planta": planta,
                    "Enlace": full_link,
                })

            except Exception as e:
                print(f"Error procesando un listado: {e}")

        # Pasar a la siguiente página
        page += 1
        time.sleep(random.uniform(1, 3))  # Añadir un retraso

    except Exception as e:
        print(f"Error procesando la página {page}: {e}")
        break

# Mostrar todas las tarjetas encontradas
print(f"Se encontraron {len(all_listings)} inmuebles.")
for listing in all_listings:
    print(listing)

Procesando página 1...
Procesando página 2...
Procesando página 3...
Procesando página 4...
Procesando página 5...
Procesando página 6...
Procesando página 7...
Procesando página 8...
Procesando página 9...
Procesando página 10...
Procesando página 11...
Procesando página 12...
Procesando página 13...
Procesando página 14...
Procesando página 15...
Procesando página 16...
Procesando página 17...
Procesando página 18...
Procesando página 19...
Procesando página 20...
Procesando página 21...
Procesando página 22...
Procesando página 23...
Procesando página 24...
Procesando página 25...
Procesando página 26...
Procesando página 27...
Procesando página 28...
Procesando página 29...
Procesando página 30...
Procesando página 31...
Procesando página 32...
Procesando página 33...
Procesando página 34...
Procesando página 35...
Procesando página 36...
Procesando página 37...
Procesando página 38...
Procesando página 39...
Procesando página 40...
Procesando página 41...
Procesando página 42...
P